# Module 4 Project

Jaklyn Soler

FlatIron School Full-Time Data Science Program

# Objective

Provided with median sale values of single family homes from Zillow, you will be responsible for recommending 5 top zip codes for investing to a real estate investment company. You will be responsible for analyzing risk, profit, and exploring other concepts independently.

# Loading Zillow Data

### Relevant packages can be imported.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib
import xlrd
from statsmodels.graphics.tsaplots import plot_pacf
from statsmodels.graphics.tsaplots import plot_acf
from pmdarima.arima import auto_arima
from pmdarima.arima import ADFTest
from sklearn.metrics import r2_score
from pmdarima.arima import ndiffs

### Truncating can be removed for better analysis. 

In [2]:
pd.set_option('display.max_columns', None) #prevents trailing elipses
pd.set_option('display.max_rows', None)

In [3]:
df = pd.read_csv('zillow_data.csv')
df.head()

,RegionID,RegionName,City,State,Metro,CountyName,SizeRank,1996-04,1996-05,1996-06,1996-07,1996-08,1996-09,1996-10,1996-11,1996-12,1997-01,1997-02,1997-03,1997-04,1997-05,1997-06,1997-07,1997-08,1997-09,1997-10,1997-11,1997-12,1998-01,1998-02,1998-03,1998-04,1998-05,1998-06,1998-07,1998-08,1998-09,1998-10,1998-11,1998-12,1999-01,1999-02,1999-03,1999-04,1999-05,1999-06,1999-07,1999-08,1999-09,1999-10,1999-11,1999-12,2000-01,2000-02,2000-03,2000-04,2000-05,2000-06,2000-07,2000-08,2000-09,2000-10,2000-11,2000-12,2001-01,2001-02,2001-03,2001-04,2001-05,2001-06,2001-07,2001-08,2001-09,2001-10,2001-11,2001-12,2002-01,2002-02,2002-03,2002-04,2002-05,2002-06,2002-07,2002-08,2002-09,2002-10,2002-11,2002-12,2003-01,2003-02,2003-03,2003-04,2003-05,2003-06,2003-07,2003-08,2003-09,2003-10,2003-11,2003-12,2004-01,2004-02,2004-03,2004-04,2004-05,2004-06,2004-07,2004-08,2004-09,2004-10,2004-11,2004-12,2005-01,2005-02,2005-03,2005-04,2005-05,2005-06,2005-07,2005-08,2005-09,2005-10,2005-11,2005-12,2006-01,2006-02,2006-03,2006-04,2006-05,2006-06,2006-07,2006-08,2006-09,2006-10,2006-11,2006-12,2007-01,2007-02,2007-03,2007-04,2007-05,2007-06,2007-07,2007-08,2007-09,2007-10,2007-11,2007-12,2008-01,2008-02,2008-03,2008-04,2008-05,2008-06,2008-07,2008-08,2008-09,2008-10,2008-11,2008-12,2009-01,2009-02,2009-03,2009-04,2009-05,2009-06,2009-07,2009-08,2009-09,2009-10,2009-11,2009-12,2010-01,2010-02,2010-03,2010-04,2010-05,2010-06,2010-07,2010-08,2010-09,2010-10,2010-11,2010-12,2011-01,2011-02,2011-03,2011-04,2011-05,2011-06,2011-07,2011-08,2011-09,2011-10,2011-11,2011-12,2012-01,2012-02,2012-03,2012-04,2012-05,2012-06,2012-07,2012-08,2012-09,2012-10,2012-11,2012-12,2013-01,2013-02,2013-03,2013-04,2013-05,2013-06,2013-07,2013-08,2013-09,2013-10,2013-11,2013-12,2014-01,2014-02,2014-03,2014-04,2014-05,2014-06,2014-07,2014-08,2014-09,2014-10,2014-11,2014-12,2015-01,2015-02,2015-03,2015-04,2015-05,2015-06,2015-07,2015-08,2015-09,2015-10,2015-11,2015-12,2016-01,2016-02,2016-03,2016-04,2016-05,2016-06,2016-07,2016-08,2016-09,2016-10,2016-11,2016-12,2017-01,2017-02,2017-03,2017-04,2017-05,2017-06,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04
0,84654,60657,Chicago,IL,Chicago,Cook,1,334200.0,335400.0,336500.0,337600.0,338500.0,339500.0,340400.0,341300.0,342600.0,344400.0,345700.0,346700.0,347800.0,349000.0,350400.0,352000.0,353900.0,356200.0,358800.0,361800.0,365700.0,370200.0,374700.0,378900.0,383500.0,388300.0,393300.0,398500.0,403800.0,409100.0,414600.0,420100.0,426200.0,432600.0,438600.0,444200.0,450000.0,455900.0,462100.0,468500.0,475300.0,482500.0,490200.0,498200.0,507200.0,516800.0,526300.0,535300.0,544500.0,553500.0,562400.0,571200.0,579800.0,588100.0,596300.0,604200.0,612200.0,620200.0,627700.0,634500.0,641000.0,647000.0,652700.0,658100.0,663300.0,668400.0,673400.0,678300.0,683200.0,688300.0,693300.0,698000.0,702400.0,706400.0,710200.0,714000.0,717800.0,721700.0,725700.0,729900.0,733400.0,735600.0,737200.0,739000.0,740900.0,742700.0,744400.0,746000.0,747200.0,748000.0,749000.0,750200.0,752300.0,755300.0,759200.0,764000.0,769600.0,775600.0,781900.0,787900.0,793200.0,798200.0,803100.0,807900.0,812900.0,818100.0,823100.0,828300.0,834100.0,839800.0,845600.0,851800.0,858000.0,864400.0,870700.0,876200.0,880700.0,884400.0,887600.0,890500.0,893300.0,895500.0,897300.0,899000.0,900400.0,902000.0,904400.0,907100.0,909700.0,911900.0,913000.0,913000.0,912000.0,909300.0,905300.0,901400.0,897900.0,895400.0,893600.0,891100.0,887000.0,881700.0,875900.0,870300.0,865100.0,859000.0,851500.0,843800.0,836400.0,830700.0,827300.0,824800.0,821600.0,818300.0,814600.0,809800.0,803600.0,795500.0,786900.0,780700.0,776900.0,774700.0,774200.0,774400.0,774600.0,775600.0,777800.0,775200.0,767900.0,764700.0,766100.0,764100.0,759700.0,754900.0,746200.0,737300.0,730800.0,729300.0,730200.0,730700.0,730000.0,730100.0,730100.0,731200.0,733900.0,735500.0,735400.0,734400.0,737500.0,737700.0,733700.0,734000.0,740300.0,744600.0,750500.0,760400.0,771800.0,780600.0,

# Data Preprocessing

### Shape can be observed.

In [4]:
df.shape

(14723, 272)

### RegionID column can be removed with SizeRank.

In [5]:
df.drop(columns=['RegionID', 'SizeRank'], inplace=True)

### NaN values can be identified.

In [6]:
df.isna().sum()

RegionName       0
City             0
State            0
Metro         1043
CountyName       0
1996-04       1039
1996-05       1039
1996-06       1039
1996-07       1039
1996-08       1039
1996-09       1039
1996-10       1039
1996-11       1039
1996-12       1039
1997-01       1039
1997-02       1039
1997-03       1039
1997-04       1039
1997-05       1039
1997-06       1039
1997-07       1038
1997-08       1038
1997-09       1038
1997-10       1038
1997-11       1038
1997-12       1038
1998-01       1036
1998-02       1036
1998-03       1036
1998-04       1036
1998-05       1036
1998-06       1036
1998-07       1036
1998-08       1036
1998-09       1036
1998-10       1036
1998-11       1036
1998-12       1036
1999-01       1036
1999-02       1036
1999-03       1036
1999-04       1036
1999-05       1036
1999-06       1036
1999-07       1036
1999-08       1036
1999-09       1036
1999-10       1036
1999-11       1036
1999-12       1036
2000-01       1036
2000-02       1036
2000-03     

In [7]:
df[:][:].isna().sum().sum()

157934

### Metro column containing NaN values can be removed. 

In [8]:
df.drop(columns='Metro', inplace = True)

In [9]:
df.head(2)

,RegionName,City,State,CountyName,1996-04,1996-05,1996-06,1996-07,1996-08,1996-09,1996-10,1996-11,1996-12,1997-01,1997-02,1997-03,1997-04,1997-05,1997-06,1997-07,1997-08,1997-09,1997-10,1997-11,1997-12,1998-01,1998-02,1998-03,1998-04,1998-05,1998-06,1998-07,1998-08,1998-09,1998-10,1998-11,1998-12,1999-01,1999-02,1999-03,1999-04,1999-05,1999-06,1999-07,1999-08,1999-09,1999-10,1999-11,1999-12,2000-01,2000-02,2000-03,2000-04,2000-05,2000-06,2000-07,2000-08,2000-09,2000-10,2000-11,2000-12,2001-01,2001-02,2001-03,2001-04,2001-05,2001-06,2001-07,2001-08,2001-09,2001-10,2001-11,2001-12,2002-01,2002-02,2002-03,2002-04,2002-05,2002-06,2002-07,2002-08,2002-09,2002-10,2002-11,2002-12,2003-01,2003-02,2003-03,2003-04,2003-05,2003-06,2003-07,2003-08,2003-09,2003-10,2003-11,2003-12,2004-01,2004-02,2004-03,2004-04,2004-05,2004-06,2004-07,2004-08,2004-09,2004-10,2004-11,2004-12,2005-01,2005-02,2005-03,2005-04,2005-05,2005-06,2005-07,2005-08,2005-09,2005-10,2005-11,2005-12,2006-01,2006-02,2006-03,2006-04,2006-05,2006-06,2006-07,2006-08,2006-09,2006-10,2006-11,2006-12,2007-01,2007-02,2007-03,2007-04,2007-05,2007-06,2007-07,2007-08,2007-09,2007-10,2007-11,2007-12,2008-01,2008-02,2008-03,2008-04,2008-05,2008-06,2008-07,2008-08,2008-09,2008-10,2008-11,2008-12,2009-01,2009-02,2009-03,2009-04,2009-05,2009-06,2009-07,2009-08,2009-09,2009-10,2009-11,2009-12,2010-01,2010-02,2010-03,2010-04,2010-05,2010-06,2010-07,2010-08,2010-09,2010-10,2010-11,2010-12,2011-01,2011-02,2011-03,2011-04,2011-05,2011-06,2011-07,2011-08,2011-09,2011-10,2011-11,2011-12,2012-01,2012-02,2012-03,2012-04,2012-05,2012-06,2012-07,2012-08,2012-09,2012-10,2012-11,2012-12,2013-01,2013-02,2013-03,2013-04,2013-05,2013-06,2013-07,2013-08,2013-09,2013-10,2013-11,2013-12,2014-01,2014-02,2014-03,2014-04,2014-05,2014-06,2014-07,2014-08,2014-09,2014-10,2014-11,2014-12,2015-01,2015-02,2015-03,2015-04,2015-05,2015-06,2015-07,2015-08,2015-09,2015-10,2015-11,2015-12,2016-01,2016-02,2016-03,2016-04,2016-05,2016-06,2016-07,2016-08,2016-09,2016-10,2016-11,2016-12,2017-01,2017-02,2017-03,2017-04,2017-05,2017-06,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04
0,60657,Chicago,IL,Cook,334200.0,335400.0,336500.0,337600.0,338500.0,339500.0,340400.0,341300.0,342600.0,344400.0,345700.0,346700.0,347800.0,349000.0,350400.0,352000.0,353900.0,356200.0,358800.0,361800.0,365700.0,370200.0,374700.0,378900.0,383500.0,388300.0,393300.0,398500.0,403800.0,409100.0,414600.0,420100.0,426200.0,432600.0,438600.0,444200.0,450000.0,455900.0,462100.0,468500.0,475300.0,482500.0,490200.0,498200.0,507200.0,516800.0,526300.0,535300.0,544500.0,553500.0,562400.0,571200.0,579800.0,588100.0,596300.0,604200.0,612200.0,620200.0,627700.0,634500.0,641000.0,647000.0,652700.0,658100.0,663300.0,668400.0,673400.0,678300.0,683200.0,688300.0,693300.0,698000.0,702400.0,706400.0,710200.0,714000.0,717800.0,721700.0,725700.0,729900.0,733400.0,735600.0,737200.0,739000.0,740900.0,742700.0,744400.0,746000.0,747200.0,748000.0,749000.0,750200.0,752300.0,755300.0,759200.0,764000.0,769600.0,775600.0,781900.0,787900.0,793200.0,798200.0,803100.0,807900.0,812900.0,818100.0,823100.0,828300.0,834100.0,839800.0,845600.0,851800.0,858000.0,864400.0,870700.0,876200.0,880700.0,884400.0,887600.0,890500.0,893300.0,895500.0,897300.0,899000.0,900400.0,902000.0,904400.0,907100.0,909700.0,911900.0,913000.0,913000.0,912000.0,909300.0,905300.0,901400.0,897900.0,895400.0,893600.0,891100.0,887000.0,881700.0,875900.0,870300.0,865100.0,859000.0,851500.0,843800.0,836400.0,830700.0,827300.0,824800.0,821600.0,818300.0,814600.0,809800.0,803600.0,795500.0,786900.0,780700.0,776900.0,774700.0,774200.0,774400.0,774600.0,775600.0,777800.0,775200.0,767900.0,764700.0,766100.0,764100.0,759700.0,754900.0,746200.0,737300.0,730800.0,729300.0,730200.0,730700.0,730000.0,730100.0,730100.0,731200.0,733900.0,735500.0,735400.0,734400.0,737500.0,737700.0,733700.0,734000.0,740300.0,744600.0,750500.0,760400.0,771800.0,780600.0,787900.0,794100.0,798900.0,802300.0,8061

### Column distributions can be observed.

In [10]:
df.describe()

,RegionName,1996-04,1996-05,1996-06,1996-07,1996-08,1996-09,1996-10,1996-11,1996-12,1997-01,1997-02,1997-03,1997-04,1997-05,1997-06,1997-07,1997-08,1997-09,1997-10,1997-11,1997-12,1998-01,1998-02,1998-03,1998-04,1998-05,1998-06,1998-07,1998-08,1998-09,1998-10,1998-11,1998-12,1999-01,1999-02,1999-03,1999-04,1999-05,1999-06,1999-07,1999-08,1999-09,1999-10,1999-11,1999-12,2000-01,2000-02,2000-03,2000-04,2000-05,2000-06,2000-07,2000-08,2000-09,2000-10,2000-11,2000-12,2001-01,2001-02,2001-03,2001-04,2001-05,2001-06,2001-07,2001-08,2001-09,2001-10,2001-11,2001-12,2002-01,2002-02,2002-03,2002-04,2002-05,2002-06,2002-07,2002-08,2002-09,2002-10,2002-11,2002-12,2003-01,2003-02,2003-03,2003-04,2003-05,2003-06,2003-07,2003-08,2003-09,2003-10,2003-11,2003-12,2004-01,2004-02,2004-03,2004-04,2004-05,2004-06,2004-07,2004-08,2004-09,2004-10,2004-11,2004-12,2005-01,2005-02,2005-03,2005-04,2005-05,2005-06,2005-07,2005-08,2005-09,2005-10,2005-11,2005-12,2006-01,2006-02,2006-03,2006-04,2006-05,2006-06,2006-07,2006-08,2006-09,2006-10,2006-11,2006-12,2007-01,2007-02,2007-03,2007-04,2007-05,2007-06,2007-07,2007-08,2007-09,2007-10,2007-11,2007-12,2008-01,2008-02,2008-03,2008-04,2008-05,2008-06,2008-07,2008-08,2008-09,2008-10,2008-11,2008-12,2009-01,2009-02,2009-03,2009-04,2009-05,2009-06,2009-07,2009-08,2009-09,2009-10,2009-11,2009-12,2010-01,2010-02,2010-03,2010-04,2010-05,2010-06,2010-07,2010-08,2010-09,2010-10,2010-11,2010-12,2011-01,2011-02,2011-03,2011-04,2011-05,2011-06,2011-07,2011-08,2011-09,2011-10,2011-11,2011-12,2012-01,2012-02,2012-03,2012-04,2012-05,2012-06,2012-07,2012-08,2012-09,2012-10,2012-11,2012-12,2013-01,2013-02,2013-03,2013-04,2013-05,2013-06,2013-07,2013-08,2013-09,2013-10,2013-11,2013-12,2014-01,2014-02,2014-03,2014-04,2014-05,2014-06,2014-07,2014-08,2014-09,2014-10,2014-11,2014-12,2015-01,2015-02,2015-03,2015-04,2015-05,2015-06,2015-07,2015-08,2015-09,2015-10,2015-11,2015-12,2016-01,2016-02,2016-03,2016-04,2016-05,2016-06,2016-07,2016-08,2016-09,2016-10,2016-11,2016-12,2017-01,2017-02,2017-03,2017-04,2017-05,2017-06,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04
count,14723.000000,1.368400e+04,1.368400e+04,1.368400e+04,1.368400e+04,1.368400e+04,1.368400e+04,1.368400e+04,1.368400e+04,1.368400e+04,1.368400e+04,1.368400e+04,1.368400e+04,1.368400e+04,1.368400e+04,1.368400e+04,1.368500e+04,1.368500e+04,1.368500e+04,1.368500e+04,1.368500e+04,1.368500e+04,1.368700e+04,1.368700e+04,1.368700e+04,1.368700e+04,1.368700e+04,1.368700e+04,1.368700e+04,1.368700e+04,1.368700e+04,1.368700e+04,1.368700e+04,1.368700e+04,1.368700e+04,1.368700e+04,1.368700e+04,1.368700e+04,1.368700e+04,1.368700e+04,1.368700e+04,1.368700e+04,1.368700e+04,1.368700e+04,1.368700e+04,1.368700e+04,1.368700e+04,1.368700e+04,1.368700e+04,1.368700e+04,1.368700e+04,1.368700e+04,1.368700e+04,1.368700e+04,1.368700e+04,1.368700e+04,1.368700e+04,1.368700e+04,1.368700e+04,1.368700e+04,1.368700e+04,1.368700e+04,1.368700e+04,1.368700e+04,1.368700e+04,1.368700e+04,1.368700e+04,1.368700e+04,1.368700e+04,1.368700e+04,1.368700e+04,1.368700e+04,1.368700e+04,1.368700e+04,1.368700e+04,1.368700e+04,1.368700e+04,1.368700e+04,1.368700e+04,1.368700e+04,1.368700e+04,1.368700e+04,1.368700e+04,1.368700e+04,1.368700e+04,1.368700e+04,1.368700e+04,1.368700e+04,1.380500e+04,1.380500e+04,1.380500e+04,1.380500e+04,1.380500e+04,1.380500e+04,1.383600e+04,1.383600e+04,1.383600e+04,1.383600e+04,1.383600e+04,1.383600e+04,1.385700e+04,1.385700e+04,1.385700e+04,1.385700e+04,1.385700e+04,1.385700e+04,1.390900e+04,1.390900e+04,1.392200e+04,1.392200e+04,1.392200e+04,1.392200e+04,1.400000e+04,1.400000e+04,1.400000e+04,1.400000e+04,1.400000e+04,1.400000e+04,1.405600e+04,1.405600e+04,1.405600e+04,1.405600e+04,1.405600e+04,1.405600e+04,1.408300e+04,1.408300e+04,1.408300e+04,1.408300e+04,1.408300e+04,1.408300e+04,1.410300e+04,1.410300e+04,1.410300e+04,1.410300e+04,1.410300e+04,1.410300e+04,1.411000e+04,1.411000e+04,1.411000e+04,1.411000e+04,1.411000e+04,1.411000e+04,1.411600e+04,1.4

### RegionName column can be renamed to Zipcode.

In [11]:
df.rename(columns={'RegionName':'Zipcode'}, inplace=True)

In [12]:
df.head()

,Zipcode,City,State,CountyName,1996-04,1996-05,1996-06,1996-07,1996-08,1996-09,1996-10,1996-11,1996-12,1997-01,1997-02,1997-03,1997-04,1997-05,1997-06,1997-07,1997-08,1997-09,1997-10,1997-11,1997-12,1998-01,1998-02,1998-03,1998-04,1998-05,1998-06,1998-07,1998-08,1998-09,1998-10,1998-11,1998-12,1999-01,1999-02,1999-03,1999-04,1999-05,1999-06,1999-07,1999-08,1999-09,1999-10,1999-11,1999-12,2000-01,2000-02,2000-03,2000-04,2000-05,2000-06,2000-07,2000-08,2000-09,2000-10,2000-11,2000-12,2001-01,2001-02,2001-03,2001-04,2001-05,2001-06,2001-07,2001-08,2001-09,2001-10,2001-11,2001-12,2002-01,2002-02,2002-03,2002-04,2002-05,2002-06,2002-07,2002-08,2002-09,2002-10,2002-11,2002-12,2003-01,2003-02,2003-03,2003-04,2003-05,2003-06,2003-07,2003-08,2003-09,2003-10,2003-11,2003-12,2004-01,2004-02,2004-03,2004-04,2004-05,2004-06,2004-07,2004-08,2004-09,2004-10,2004-11,2004-12,2005-01,2005-02,2005-03,2005-04,2005-05,2005-06,2005-07,2005-08,2005-09,2005-10,2005-11,2005-12,2006-01,2006-02,2006-03,2006-04,2006-05,2006-06,2006-07,2006-08,2006-09,2006-10,2006-11,2006-12,2007-01,2007-02,2007-03,2007-04,2007-05,2007-06,2007-07,2007-08,2007-09,2007-10,2007-11,2007-12,2008-01,2008-02,2008-03,2008-04,2008-05,2008-06,2008-07,2008-08,2008-09,2008-10,2008-11,2008-12,2009-01,2009-02,2009-03,2009-04,2009-05,2009-06,2009-07,2009-08,2009-09,2009-10,2009-11,2009-12,2010-01,2010-02,2010-03,2010-04,2010-05,2010-06,2010-07,2010-08,2010-09,2010-10,2010-11,2010-12,2011-01,2011-02,2011-03,2011-04,2011-05,2011-06,2011-07,2011-08,2011-09,2011-10,2011-11,2011-12,2012-01,2012-02,2012-03,2012-04,2012-05,2012-06,2012-07,2012-08,2012-09,2012-10,2012-11,2012-12,2013-01,2013-02,2013-03,2013-04,2013-05,2013-06,2013-07,2013-08,2013-09,2013-10,2013-11,2013-12,2014-01,2014-02,2014-03,2014-04,2014-05,2014-06,2014-07,2014-08,2014-09,2014-10,2014-11,2014-12,2015-01,2015-02,2015-03,2015-04,2015-05,2015-06,2015-07,2015-08,2015-09,2015-10,2015-11,2015-12,2016-01,2016-02,2016-03,2016-04,2016-05,2016-06,2016-07,2016-08,2016-09,2016-10,2016-11,2016-12,2017-01,2017-02,2017-03,2017-04,2017-05,2017-06,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04
0,60657,Chicago,IL,Cook,334200.0,335400.0,336500.0,337600.0,338500.0,339500.0,340400.0,341300.0,342600.0,344400.0,345700.0,346700.0,347800.0,349000.0,350400.0,352000.0,353900.0,356200.0,358800.0,361800.0,365700.0,370200.0,374700.0,378900.0,383500.0,388300.0,393300.0,398500.0,403800.0,409100.0,414600.0,420100.0,426200.0,432600.0,438600.0,444200.0,450000.0,455900.0,462100.0,468500.0,475300.0,482500.0,490200.0,498200.0,507200.0,516800.0,526300.0,535300.0,544500.0,553500.0,562400.0,571200.0,579800.0,588100.0,596300.0,604200.0,612200.0,620200.0,627700.0,634500.0,641000.0,647000.0,652700.0,658100.0,663300.0,668400.0,673400.0,678300.0,683200.0,688300.0,693300.0,698000.0,702400.0,706400.0,710200.0,714000.0,717800.0,721700.0,725700.0,729900.0,733400.0,735600.0,737200.0,739000.0,740900.0,742700.0,744400.0,746000.0,747200.0,748000.0,749000.0,750200.0,752300.0,755300.0,759200.0,764000.0,769600.0,775600.0,781900.0,787900.0,793200.0,798200.0,803100.0,807900.0,812900.0,818100.0,823100.0,828300.0,834100.0,839800.0,845600.0,851800.0,858000.0,864400.0,870700.0,876200.0,880700.0,884400.0,887600.0,890500.0,893300.0,895500.0,897300.0,899000.0,900400.0,902000.0,904400.0,907100.0,909700.0,911900.0,913000.0,913000.0,912000.0,909300.0,905300.0,901400.0,897900.0,895400.0,893600.0,891100.0,887000.0,881700.0,875900.0,870300.0,865100.0,859000.0,851500.0,843800.0,836400.0,830700.0,827300.0,824800.0,821600.0,818300.0,814600.0,809800.0,803600.0,795500.0,786900.0,780700.0,776900.0,774700.0,774200.0,774400.0,774600.0,775600.0,777800.0,775200.0,767900.0,764700.0,766100.0,764100.0,759700.0,754900.0,746200.0,737300.0,730800.0,729300.0,730200.0,730700.0,730000.0,730100.0,730100.0,731200.0,733900.0,735500.0,735400.0,734400.0,737500.0,737700.0,733700.0,734000.0,740300.0,744600.0,750500.0,760400.0,771800.0,780600.0,787900.0,794100.0,798900.0,802300.0,806100.

# EDA and Visualization

In [13]:
df.State.value_counts()

CA    1224
NY    1015
TX     989
PA     831
FL     785
OH     588
IL     547
NJ     502
MI     499
IN     428
NC     428
MA     417
TN     404
VA     401
MN     375
GA     345
WA     341
WI     332
MO     319
MD     317
CO     249
KS     241
AZ     230
OR     224
OK     221
SC     206
NH     199
LA     193
AL     183
IA     158
MS     153
KY     139
CT     124
UT     121
ID     110
AR     105
NV     103
ME      86
NE      83
WV      72
MT      71
HI      62
NM      60
RI      59
DE      41
WY      31
ND      31
AK      28
SD      19
DC      18
VT      16
Name: State, dtype: int64

In [14]:
df.head()

,Zipcode,City,State,CountyName,1996-04,1996-05,1996-06,1996-07,1996-08,1996-09,1996-10,1996-11,1996-12,1997-01,1997-02,1997-03,1997-04,1997-05,1997-06,1997-07,1997-08,1997-09,1997-10,1997-11,1997-12,1998-01,1998-02,1998-03,1998-04,1998-05,1998-06,1998-07,1998-08,1998-09,1998-10,1998-11,1998-12,1999-01,1999-02,1999-03,1999-04,1999-05,1999-06,1999-07,1999-08,1999-09,1999-10,1999-11,1999-12,2000-01,2000-02,2000-03,2000-04,2000-05,2000-06,2000-07,2000-08,2000-09,2000-10,2000-11,2000-12,2001-01,2001-02,2001-03,2001-04,2001-05,2001-06,2001-07,2001-08,2001-09,2001-10,2001-11,2001-12,2002-01,2002-02,2002-03,2002-04,2002-05,2002-06,2002-07,2002-08,2002-09,2002-10,2002-11,2002-12,2003-01,2003-02,2003-03,2003-04,2003-05,2003-06,2003-07,2003-08,2003-09,2003-10,2003-11,2003-12,2004-01,2004-02,2004-03,2004-04,2004-05,2004-06,2004-07,2004-08,2004-09,2004-10,2004-11,2004-12,2005-01,2005-02,2005-03,2005-04,2005-05,2005-06,2005-07,2005-08,2005-09,2005-10,2005-11,2005-12,2006-01,2006-02,2006-03,2006-04,2006-05,2006-06,2006-07,2006-08,2006-09,2006-10,2006-11,2006-12,2007-01,2007-02,2007-03,2007-04,2007-05,2007-06,2007-07,2007-08,2007-09,2007-10,2007-11,2007-12,2008-01,2008-02,2008-03,2008-04,2008-05,2008-06,2008-07,2008-08,2008-09,2008-10,2008-11,2008-12,2009-01,2009-02,2009-03,2009-04,2009-05,2009-06,2009-07,2009-08,2009-09,2009-10,2009-11,2009-12,2010-01,2010-02,2010-03,2010-04,2010-05,2010-06,2010-07,2010-08,2010-09,2010-10,2010-11,2010-12,2011-01,2011-02,2011-03,2011-04,2011-05,2011-06,2011-07,2011-08,2011-09,2011-10,2011-11,2011-12,2012-01,2012-02,2012-03,2012-04,2012-05,2012-06,2012-07,2012-08,2012-09,2012-10,2012-11,2012-12,2013-01,2013-02,2013-03,2013-04,2013-05,2013-06,2013-07,2013-08,2013-09,2013-10,2013-11,2013-12,2014-01,2014-02,2014-03,2014-04,2014-05,2014-06,2014-07,2014-08,2014-09,2014-10,2014-11,2014-12,2015-01,2015-02,2015-03,2015-04,2015-05,2015-06,2015-07,2015-08,2015-09,2015-10,2015-11,2015-12,2016-01,2016-02,2016-03,2016-04,2016-05,2016-06,2016-07,2016-08,2016-09,2016-10,2016-11,2016-12,2017-01,2017-02,2017-03,2017-04,2017-05,2017-06,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04
0,60657,Chicago,IL,Cook,334200.0,335400.0,336500.0,337600.0,338500.0,339500.0,340400.0,341300.0,342600.0,344400.0,345700.0,346700.0,347800.0,349000.0,350400.0,352000.0,353900.0,356200.0,358800.0,361800.0,365700.0,370200.0,374700.0,378900.0,383500.0,388300.0,393300.0,398500.0,403800.0,409100.0,414600.0,420100.0,426200.0,432600.0,438600.0,444200.0,450000.0,455900.0,462100.0,468500.0,475300.0,482500.0,490200.0,498200.0,507200.0,516800.0,526300.0,535300.0,544500.0,553500.0,562400.0,571200.0,579800.0,588100.0,596300.0,604200.0,612200.0,620200.0,627700.0,634500.0,641000.0,647000.0,652700.0,658100.0,663300.0,668400.0,673400.0,678300.0,683200.0,688300.0,693300.0,698000.0,702400.0,706400.0,710200.0,714000.0,717800.0,721700.0,725700.0,729900.0,733400.0,735600.0,737200.0,739000.0,740900.0,742700.0,744400.0,746000.0,747200.0,748000.0,749000.0,750200.0,752300.0,755300.0,759200.0,764000.0,769600.0,775600.0,781900.0,787900.0,793200.0,798200.0,803100.0,807900.0,812900.0,818100.0,823100.0,828300.0,834100.0,839800.0,845600.0,851800.0,858000.0,864400.0,870700.0,876200.0,880700.0,884400.0,887600.0,890500.0,893300.0,895500.0,897300.0,899000.0,900400.0,902000.0,904400.0,907100.0,909700.0,911900.0,913000.0,913000.0,912000.0,909300.0,905300.0,901400.0,897900.0,895400.0,893600.0,891100.0,887000.0,881700.0,875900.0,870300.0,865100.0,859000.0,851500.0,843800.0,836400.0,830700.0,827300.0,824800.0,821600.0,818300.0,814600.0,809800.0,803600.0,795500.0,786900.0,780700.0,776900.0,774700.0,774200.0,774400.0,774600.0,775600.0,777800.0,775200.0,767900.0,764700.0,766100.0,764100.0,759700.0,754900.0,746200.0,737300.0,730800.0,729300.0,730200.0,730700.0,730000.0,730100.0,730100.0,731200.0,733900.0,735500.0,735400.0,734400.0,737500.0,737700.0,733700.0,734000.0,740300.0,744600.0,750500.0,760400.0,771800.0,780600.0,787900.0,794100.0,798900.0,802300.0,806100.

# Reshape from Wide to Long Format

### The missing values will be dropped.

In [15]:
def melt_data(df):
    melted = pd.melt(df, id_vars=['Zipcode', 'City', 'State', 'CountyName'], var_name='time')
    melted['time'] = pd.to_datetime(melted['time'], infer_datetime_format=True)
    melted = melted.dropna(subset=['value'])
    return melted #.groupby('time').aggregate({'value':'mean'})

In [ ]:
melted = melt_data(df)

In [ ]:
melted.head()

### Data before 2012 can be removed to account for the housing crisis.

In [18]:
# years removal here

In [19]:
melted.set_index('time', inplace = True)

In [20]:
post_recession = melted.groupby(['Zipcode'], as_index=True).resample('M').sum()
#post_recession.reset_index(inplace = True)
#post_recession = post_recession.sort_index().loc['2018-12-31': '2012-01-01']

In [21]:
post_recession.head()

Zipcode     value
Zipcode time                         
1001    1996-04-30     1001  113100.0
        1996-05-31     1001  112800.0
        1996-06-30     1001  112600.0
        1996-07-31     1001  112300.0
        1996-08-31     1001  112100.0

In [ ]:
post_recession.set_index('time')

In [34]:
post_recession['2018-12-31': '2012-01-01']

AttributeError: 'function' object has no attribute 'iloc'

In [22]:
melted.isna().sum()

Zipcode       0
City          0
State         0
CountyName    0
value         0
dtype: int64

In [23]:
melted.dtypes

Zipcode         int64
City           object
State          object
CountyName     object
value         float64
dtype: object

# Narrowing Down Geographic Area

### Due to the high number of baby boomers living in new york where demand for property is high based on population, we will continue with that dataset since it will likely result in a high amount of probate properties as time goes on. 

In [24]:
ny = melted[melted['State'] == 'NY']

In [25]:
ny.head()

,Zipcode,City,State,CountyName,value
time,,,,,
1996-04-01,10467,New York,NY,Bronx,152900.0
1996-04-01,11226,New York,NY,Kings,162000.0
1996-04-01,11375,New York,NY,Queens,252400.0
1996-04-01,11235,New York,NY,Kings,190500.0
1996-04-01,10128,New York,NY,New York,3676700.0


In [26]:
ny.Zipcode.value_counts().sort_index()

10011    160
10014    172
10021    172
10128    265
10301    265
10302    265
10303    265
10304    265
10305    265
10306    265
10307    265
10308    265
10309    265
10310    265
10312    265
10314    265
10456     52
10459    178
10461    265
10462    265
10463    265
10464    265
10465    265
10466    265
10467    265
10469    265
10470    265
10471    265
10473    265
10501    265
10502    265
10504    265
10506    265
10507    265
10509    265
10510    265
10512    265
10514    265
10516    265
10518    265
10520    265
10522    265
10523    265
10524    265
10526    265
10528    265
10530    265
10532    265
10533    265
10536    265
10537    265
10538    265
10541    265
10543    265
10546    265
10547    265
10549    265
10550     94
10552    265
10553    265
10560    265
10562    265
10566    265
10567    265
10570    265
10573    265
10576    265
10577    265
10579    265
10580    265
10583    265
10588    265
10589    265
10590    265
10591    265
10594    265
10595    265

### properties under 800k. Additionally we want to choose a zipcode with 265 data points to ensure the validity of the findings. 

In [27]:
ny_under = ny[ny['value'] <= 800000]

In [28]:
ny_under = ny[ny['value'] >= 400000]

In [29]:
ny_under.head()

,Zipcode,City,State,CountyName,value
time,,,,,
1996-04-01,10128,New York,NY,New York,3676700.0
1996-04-01,10583,Scarsdale,NY,Westchester,407700.0
1996-04-01,11021,Great Neck,NY,Nassau,449000.0
1996-04-01,10538,Larchmont,NY,Westchester,407200.0
1996-04-01,10580,Rye,NY,Westchester,491700.0


In [30]:
ny_under.Zipcode.value_counts()

11021    265
11030    265
10514    265
11020    265
11724    265
11576    265
11568    265
11023    265
11024    265
10528    265
10128    265
10577    265
10576    265
10583    265
10504    265
10580    265
11797    265
11976    265
10506    265
10538    265
11975    254
11559    245
11545    243
11959    237
11932    232
11771    231
10533    230
10510    230
11577    230
10471    228
11363    227
11930    227
11050    227
10549    225
11530    224
10708    223
10804    222
10590    222
11694    221
10968    221
11753    220
11360    220
11215    220
10536    220
11560    220
10518    219
11509    219
10803    217
10501    216
10605    216
11598    213
11362    213
10706    212
11579    210
10546    210
11968    210
11768    209
14722    209
10560    209
11791    209
10964    208
10543    208
10522    208
10502    208
11569    206
11937    206
11963    206
11732    206
11777    205
10709    204
10570    204
11230    204
11209    203
11977    202
11739    202
10604    202
11364    201

In [31]:
ny_under.set_index('time', inplace = True)

KeyError: "None of ['time'] are in the columns"

### The zipcodes that are being taken under consideration: 

 - 11021
 - 11030
 - 10514
 - 11020
 - 11724
 - 11576
 - 11568
 - 11023
 - 11024
 - 10528
 - 10128
 - 10577
 - 10576
 - 10583
 - 10504
 - 10580
 - 11797
 - 11976
 - 10506
 - 10538

### Separate dataframes can be created for each zipcode.

### Modeling for each one 

In [ ]:
oh_two_one = ny_under[ny_under['Zipcode'] == 11021]

In [ ]:
oh_two_one.head()

In [ ]:
oh_two_one.drop(columns = ['Zipcode', 'City', 'State', 'CountyName'], inplace = True)

In [ ]:
oh_three_oh = ny_under[ny_under['Zipcode'] == 11030]

In [ ]:
oh_three_oh.drop(columns = ['Zipcode', 'City', 'State', 'CountyName'], inplace = True)

In [ ]:
five_one_four = ny_under[ny_under['Zipcode'] == 10514]

In [ ]:
five_one_four.drop(columns = ['Zipcode', 'City', 'State', 'CountyName'], inplace = True)

In [ ]:
oh_two_oh = ny_under[ny_under['Zipcode'] == 11020]

In [ ]:
oh_two_oh.drop(columns = ['Zipcode', 'City', 'State', 'CountyName'], inplace = True)

In [ ]:
seven_two_four  = ny_under[ny_under['Zipcode'] == 11724]

In [ ]:
seven_two_four.drop(columns = ['Zipcode', 'City', 'State', 'CountyName'], inplace = True)

In [ ]:
five_seven_six = ny_under[ny_under['Zipcode'] == 11576]

In [ ]:
five_seven_six.drop(columns = ['Zipcode', 'City', 'State', 'CountyName'], inplace = True)

In [ ]:
five_six_eight = ny_under[ny_under['Zipcode'] == 11568]

In [ ]:
five_six_eight.drop(columns = ['Zipcode', 'City', 'State', 'CountyName'], inplace = True)

In [ ]:
oh_two_three = ny_under[ny_under['Zipcode'] == 11023]

In [ ]:
oh_two_three.drop(columns = ['Zipcode', 'City', 'State', 'CountyName'], inplace = True)

In [ ]:
oh_two_four = ny_under[ny_under['Zipcode'] == 11024]

In [ ]:
oh_two_four.drop(columns = ['Zipcode', 'City', 'State', 'CountyName'], inplace = True)

In [ ]:
five_two_eight = ny_under[ny_under['Zipcode'] == 10528]

In [ ]:
five_two_eight.drop(columns = ['Zipcode', 'City', 'State', 'CountyName'], inplace = True)

In [ ]:
one_two_eight = ny_under[ny_under['Zipcode'] == 10128]

In [ ]:
one_two_eight.drop(columns = ['Zipcode', 'City', 'State', 'CountyName'], inplace = True)

In [ ]:
five_seven_seven = ny_under[ny_under['Zipcode'] == 10577]

In [ ]:
five_seven_seven.drop(columns = ['Zipcode', 'City', 'State', 'CountyName'], inplace = True)

In [ ]:
oh_five_seven_six = ny_under[ny_under['Zipcode'] == 10576]

In [ ]:
oh_five_seven_six.drop(columns = ['Zipcode', 'City', 'State', 'CountyName'], inplace = True)

In [ ]:
five_eight_three = ny_under[ny_under['Zipcode'] == 10583]

In [ ]:
five_eight_three.drop(columns = ['Zipcode', 'City', 'State', 'CountyName'], inplace = True)

In [ ]:
five_oh_four = ny_under[ny_under['Zipcode'] == 10504]

In [ ]:
five_oh_four.drop(columns = ['Zipcode', 'City', 'State', 'CountyName'], inplace = True)

In [ ]:
five_eight_oh = ny_under[ny_under['Zipcode'] == 10580]

In [ ]:
five_eight_oh.drop(columns = ['Zipcode', 'City', 'State', 'CountyName'], inplace = True)

In [ ]:
seven_nine_seven = ny_under[ny_under['Zipcode'] == 11797]

In [ ]:
seven_nine_seven.drop(columns = ['Zipcode', 'City', 'State', 'CountyName'], inplace = True)

In [ ]:
nine_seven_six = ny_under[ny_under['Zipcode'] == 11976]

In [ ]:
nine_seven_six.drop(columns = ['Zipcode', 'City', 'State', 'CountyName'], inplace = True)

In [ ]:
five_oh_six = ny_under[ny_under['Zipcode'] == 10506]

In [ ]:
five_oh_six.drop(columns = ['Zipcode', 'City', 'State', 'CountyName'], inplace = True)

In [ ]:
five_three_eight = ny_under[ny_under['Zipcode'] == 10538]

In [ ]:
five_three_eight.drop(columns = ['Zipcode', 'City', 'State', 'CountyName'], inplace = True)

### The zipcodes can be analyzed geographically.

In [ ]:
ny_under # look at a map here

# ARIMA Modeling 20 New York State Zipcodes

### Autocorrelation can be observed. 

In [ ]:
ny_under.corr()

### The housing bubble is evident in these models.

### 11021

In [ ]:
oh_two_one.plot(figsize=(18,6))
plt.xlabel('Time', fontsize=14);
plt.title('Median Sale Prices in 11021')

### The Augmented Dickey-Fuller Test can be used to test for stationarity. 

In [ ]:
oh_two_one.head()

The data is stationary. 

In [ ]:
adf_test = ADFTest(alpha = .05)
adf_test.should_diff(oh_two_one)

Auto ARIMA reduces the need to provide p, d, and q values. 

In [ ]:
train = oh_two_one[:212]
test = oh_two_one[213:265]
plt.plot(train)
plt.plot(test)

In [ ]:
from pmdarima.arima import ndiffs

In [ ]:
ndiffs(oh_two_one)

In [ ]:
arima_model = auto_arima(train, start_p = 0, d=1, start_q = 0,
                         max_p=10, max_d=10, max_q = 10, start_P = 0,
                        D = 1, start_Q=0, max_P=10, max_D = 10,
                        max_Q=5, m=12, seasonal=True, 
                        error_action = 'warn', trace = True,
                        random_state = 20, n_fits = 50)

In [ ]:
arima_model.summary()

### Forecasting can be done one the test data. 

In [ ]:
prediction = pd.DataFrame(arima_model.predict(n_periods = 52), index=test.index)
prediction.columns = ['prediction_sales']
prediction.head()

In [ ]:
plt.figure(figsize=(8,5))
plt.plot(train, label = 'Training')
plt.plot(test, label = 'Test')
plt.plot(prediction, label = 'Prediction')
plt.legend(loc = 'best')
plt.show()

In [ ]:
test['predicted_sales'] = prediction
r2_score(test['value'], test['predicted_sales'])

### 11030

In [ ]:
oh_three_oh.plot(figsize = (18,6))
plt.xlabel('Time', fontsize=14);
plt.title('Median Sale Prices in 11030')

### The Augmented Dickey-Fuller Test can be used to test for stationarity. 

In [ ]:
adf_test.should_diff(oh_three_oh)

In [ ]:
train = oh_three_oh[:212]
test = oh_three_oh[213:265]
plt.plot(train)
plt.plot(test)

In [ ]:
ndiffs(oh_two_one)

In [ ]:
arima_model = auto_arima(train, start_p = 0, d=1, start_q = 0,
                         max_p=10, max_d=10, max_q = 10, start_P = 0,
                        D = 1, start_Q=0, max_P=10, max_D = 10,
                        max_Q=5, m=12, seasonal=True, 
                        error_action = 'warn', trace = True,
                        random_state = 20, n_fits = 50)

In [ ]:
arima_model.summary()

In [ ]:
prediction = pd.DataFrame(arima_model.predict(n_periods = 52), index=test.index)
prediction.columns = ['prediction_sales']
prediction.head()

In [ ]:
plt.figure(figsize=(8,5))
plt.plot(train, label = 'Training')
plt.plot(test, label = 'Test')
plt.plot(prediction, label = 'Prediction')
plt.legend(loc = 'best')
plt.show()

In [ ]:
test['predicted_sales'] = prediction
r2_score(test['value'], test['predicted_sales'])

### 10514

In [ ]:
five_one_four.plot(figsize = (18,6))
plt.xlabel('Time', fontsize=14);
plt.title('Median Sale Prices in 10514')

### The Augmented Dickey-Fuller Test can be used to test for stationarity. 

In [ ]:
adf_test.should_diff(five_one_four)

In [ ]:
train = five_one_four[:212]
test = five_one_four[213:265]
plt.plot(train)
plt.plot(test)

In [ ]:
ndiffs(five_one_four)

In [ ]:
arima_model = auto_arima(train, start_p = 0, d=1, start_q = 0,
                         max_p=10, max_d=10, max_q = 10, start_P = 0,
                        D = 1, start_Q=0, max_P=10, max_D = 10,
                        max_Q=5, m=12, seasonal=True, 
                        error_action = 'warn', trace = True,
                        random_state = 20, n_fits = 50)

In [ ]:
arima_model.summary()

In [ ]:
prediction = pd.DataFrame(arima_model.predict(n_periods = 52), index=test.index)
prediction.columns = ['prediction_sales']
prediction.head()

In [ ]:
plt.figure(figsize=(8,5))
plt.plot(train, label = 'Training')
plt.plot(test, label = 'Test')
plt.plot(prediction, label = 'Prediction')
plt.legend(loc = 'best')
plt.show()

In [ ]:
test['predicted_sales'] = prediction
r2_score(test['value'], test['predicted_sales'])

### 11020

In [ ]:
oh_two_oh.plot(figsize = (18,6))
plt.xlabel('Time', fontsize=14);
plt.title('Median Sale Prices in 11020')

### The Augmented Dickey-Fuller Test can be used to test for stationarity. 

In [ ]:
adf_test.should_diff(oh_two_oh)

In [ ]:
train = oh_two_oh[:212]
test = oh_two_oh[213:265]
plt.plot(train)
plt.plot(test)

In [ ]:
ndiffs(oh_two_oh)

In [ ]:
arima_model = auto_arima(train, start_p = 0, d=1, start_q = 0,
                         max_p=10, max_d=10, max_q = 10, start_P = 0,
                        D = 1, start_Q=0, max_P=10, max_D = 10,
                        max_Q=5, m=12, seasonal=True, 
                        error_action = 'warn', trace = True,
                        random_state = 20, n_fits = 50)

In [ ]:
arima_model.summary()

In [ ]:
prediction = pd.DataFrame(arima_model.predict(n_periods = 52), index=test.index)
prediction.columns = ['prediction_sales']
prediction.head()

In [ ]:
plt.figure(figsize=(8,5))
plt.plot(train, label = 'Training')
plt.plot(test, label = 'Test')
plt.plot(prediction, label = 'Prediction')
plt.legend(loc = 'best')
plt.show()

In [ ]:
test['predicted_sales'] = prediction
r2_score(test['value'], test['predicted_sales'])

### 11724

In [ ]:
seven_two_four.plot(figsize = (18,6))
plt.xlabel('Time', fontsize=14);
plt.title('Median Sale Prices in 11724')

### The Augmented Dickey-Fuller Test can be used to test for stationarity. 

In [ ]:
adf_test.should_diff(seven_two_four)

In [ ]:
train = seven_two_four[:212]
test = seven_two_four[213:265]
plt.plot(train)
plt.plot(test)

In [ ]:
ndiffs(oh_two_one)

In [ ]:
arima_model = auto_arima(train, start_p = 0, d=1, start_q = 0,
                         max_p=10, max_d=10, max_q = 10, start_P = 0,
                        D = 1, start_Q=0, max_P=10, max_D = 10,
                        max_Q=5, m=12, seasonal=True, 
                        error_action = 'warn', trace = True,
                        random_state = 20, n_fits = 50)

In [ ]:
arima_model.summary()

In [ ]:
prediction = pd.DataFrame(arima_model.predict(n_periods = 52), index=test.index)
prediction.columns = ['prediction_sales']
prediction.head()

In [ ]:
plt.figure(figsize=(8,5))
plt.plot(train, label = 'Training')
plt.plot(test, label = 'Test')
plt.plot(prediction, label = 'Prediction')
plt.legend(loc = 'best')
plt.show()

In [ ]:
test['predicted_sales'] = prediction
r2_score(test['value'], test['predicted_sales'])

### 11576

In [ ]:
five_seven_six.plot(figsize = (18,6))
plt.xlabel('Time', fontsize=14);
plt.title('Median Sale Prices in 11576')

### The Augmented Dickey-Fuller Test can be used to test for stationarity. 

In [ ]:
adf_test.should_diff(five_seven_six)

In [ ]:
train = five_seven_six[:212]
test = five_seven_six[213:265]
plt.plot(train)
plt.plot(test)

In [ ]:
ndiffs(five_seven_six)

In [ ]:
arima_model = auto_arima(train, start_p = 0, d=1, start_q = 0,
                         max_p=10, max_d=10, max_q = 10, start_P = 0,
                        D = 1, start_Q=0, max_P=10, max_D = 10,
                        max_Q=5, m=12, seasonal=True, 
                        error_action = 'warn', trace = True,
                        random_state = 20, n_fits = 50)

In [ ]:
arima_model.summary()

In [ ]:
prediction = pd.DataFrame(arima_model.predict(n_periods = 52), index=test.index)
prediction.columns = ['prediction_sales']
prediction.head()

In [ ]:
plt.figure(figsize=(8,5))
plt.plot(train, label = 'Training')
plt.plot(test, label = 'Test')
plt.plot(prediction, label = 'Prediction')
plt.legend(loc = 'best')
plt.show()

In [ ]:
test['predicted_sales'] = prediction
r2_score(test['value'], test['predicted_sales'])

### 11568

In [ ]:
five_six_eight.plot(figsize = (18,6))
plt.xlabel('Time', fontsize=14);
plt.title('Median Sale Prices in 11568')

### The Augmented Dickey-Fuller Test can be used to test for stationarity. 

In [ ]:
adf_test.should_diff(five_six_eight)

In [ ]:
train = five_six_eight[:212]
test = five_six_eight[213:265]
plt.plot(train)
plt.plot(test)

In [ ]:
ndiffs(five_six_eight)

In [ ]:
arima_model = auto_arima(train, start_p = 0, d=1, start_q = 0,
                         max_p=10, max_d=10, max_q = 10, start_P = 0,
                        D = 1, start_Q=0, max_P=10, max_D = 10,
                        max_Q=5, m=12, seasonal=True, 
                        error_action = 'warn', trace = True,
                        random_state = 20, n_fits = 50)

In [ ]:
arima_model.summary()

In [ ]:
prediction = pd.DataFrame(arima_model.predict(n_periods = 52), index=test.index)
prediction.columns = ['prediction_sales']
prediction.head()

In [ ]:
plt.figure(figsize=(8,5))
plt.plot(train, label = 'Training')
plt.plot(test, label = 'Test')
plt.plot(prediction, label = 'Prediction')
plt.legend(loc = 'best')
plt.show()

In [ ]:
test['predicted_sales'] = prediction
r2_score(test['value'], test['predicted_sales'])

### 11023

In [ ]:
oh_two_three.plot(figsize = (18,6))
plt.xlabel('Time', fontsize=14);
plt.title('Median Sale Prices in 11023')

### The Augmented Dickey-Fuller Test can be used to test for stationarity. 

In [ ]:
adf_test.should_diff(oh_two_three)

In [ ]:
train = oh_two_three[:212]
test = oh_two_three[213:265]
plt.plot(train)
plt.plot(test)

In [ ]:
ndiffs(oh_two_three)

In [ ]:
arima_model = auto_arima(train, start_p = 0, d=1, start_q = 0,
                         max_p=10, max_d=10, max_q = 10, start_P = 0,
                        D = 1, start_Q=0, max_P=10, max_D = 10,
                        max_Q=5, m=12, seasonal=True, 
                        error_action = 'warn', trace = True,
                        random_state = 20, n_fits = 50)

In [ ]:
arima_model.summary()

In [ ]:
prediction = pd.DataFrame(arima_model.predict(n_periods = 52), index=test.index)
prediction.columns = ['prediction_sales']
prediction.head()

In [ ]:
plt.figure(figsize=(8,5))
plt.plot(train, label = 'Training')
plt.plot(test, label = 'Test')
plt.plot(prediction, label = 'Prediction')
plt.legend(loc = 'best')
plt.show()

In [ ]:
test['predicted_sales'] = prediction
r2_score(test['value'], test['predicted_sales'])

### 11024

In [ ]:
oh_two_four.plot(figsize = (18,6))
plt.xlabel('Time', fontsize=14);
plt.title('Median Sale Prices in 11024')

### The Augmented Dickey-Fuller Test can be used to test for stationarity. 

In [ ]:
adf_test.should_diff(oh_two_four)

In [ ]:
train = oh_two_four[:212]
test = oh_two_four[213:265]
plt.plot(train)
plt.plot(test)

In [ ]:
ndiffs(oh_two_four)

In [ ]:
arima_model = auto_arima(train, start_p = 0, d=1, start_q = 0,
                         max_p=10, max_d=10, max_q = 10, start_P = 0,
                        D = 1, start_Q=0, max_P=10, max_D = 10,
                        max_Q=5, m=12, seasonal=True, 
                        error_action = 'warn', trace = True,
                        random_state = 20, n_fits = 50)

In [ ]:
arima_model.summary()

In [ ]:
prediction = pd.DataFrame(arima_model.predict(n_periods = 52), index=test.index)
prediction.columns = ['prediction_sales']
prediction.head()

In [ ]:
plt.figure(figsize=(8,5))
plt.plot(train, label = 'Training')
plt.plot(test, label = 'Test')
plt.plot(prediction, label = 'Prediction')
plt.legend(loc = 'best')
plt.show()

In [ ]:
test['predicted_sales'] = prediction
r2_score(test['value'], test['predicted_sales'])

### 10528

In [ ]:
five_two_eight.plot(figsize = (18,6))
plt.xlabel('Time', fontsize=14);
plt.title('Median Sale Prices in 10528')

### The Augmented Dickey-Fuller Test can be used to test for stationarity. 

In [ ]:
adf_test.should_diff(five_two_eight)

In [ ]:
train = five_two_eight[:212]
test = five_two_eight[213:265]
plt.plot(train)
plt.plot(test)

In [ ]:
ndiffs(five_two_eight)

In [ ]:
arima_model = auto_arima(train, start_p = 0, d=1, start_q = 0,
                         max_p=10, max_d=10, max_q = 10, start_P = 0,
                        D = 1, start_Q=0, max_P=10, max_D = 10,
                        max_Q=5, m=12, seasonal=True, 
                        error_action = 'warn', trace = True,
                        random_state = 20, n_fits = 50)

In [ ]:
arima_model.summary()

In [ ]:
prediction = pd.DataFrame(arima_model.predict(n_periods = 52), index=test.index)
prediction.columns = ['prediction_sales']
prediction.head()

In [ ]:
plt.figure(figsize=(8,5))
plt.plot(train, label = 'Training')
plt.plot(test, label = 'Test')
plt.plot(prediction, label = 'Prediction')
plt.legend(loc = 'best')
plt.show()

In [ ]:
test['predicted_sales'] = prediction
r2_score(test['value'], test['predicted_sales'])

### 10128

In [ ]:
one_two_eight.plot(figsize = (18,6))
plt.xlabel('Time', fontsize=14);
plt.title('Median Sale Prices in 10128')

### The Augmented Dickey-Fuller Test can be used to test for stationarity. 

In [ ]:
adf_test.should_diff(one_two_eight)

In [ ]:
train = one_two_eight[:212]
test = one_two_eight[213:265]
plt.plot(train)
plt.plot(test)

In [ ]:
ndiffs(one_two_eight)

In [ ]:
arima_model = auto_arima(train, start_p = 0, d=1, start_q = 0,
                         max_p=10, max_d=10, max_q = 10, start_P = 0,
                        D = 1, start_Q=0, max_P=10, max_D = 10,
                        max_Q=5, m=12, seasonal=True, 
                        error_action = 'warn', trace = True,
                        random_state = 20, n_fits = 50)

In [ ]:
arima_model.summary()

In [ ]:
prediction = pd.DataFrame(arima_model.predict(n_periods = 52), index=test.index)
prediction.columns = ['prediction_sales']
prediction.head()

In [ ]:
plt.figure(figsize=(8,5))
plt.plot(train, label = 'Training')
plt.plot(test, label = 'Test')
plt.plot(prediction, label = 'Prediction')
plt.legend(loc = 'best')
plt.show()

In [ ]:
test['predicted_sales'] = prediction
r2_score(test['value'], test['predicted_sales'])

### 10577

In [ ]:
five_seven_seven.plot(figsize = (18,6))
plt.xlabel('Time', fontsize=14);
plt.title('Median Sale Prices in 10577')

### The Augmented Dickey-Fuller Test can be used to test for stationarity. 

In [ ]:
adf_test.should_diff(five_seven_seven)

In [ ]:
train = five_seven_seven[:212]
test = five_seven_seven[213:265]
plt.plot(train)
plt.plot(test)

In [ ]:
ndiffs(five_seven_seven)

In [ ]:
arima_model = auto_arima(train, start_p = 0, d=1, start_q = 0,
                         max_p=10, max_d=10, max_q = 10, start_P = 0,
                        D = 1, start_Q=0, max_P=10, max_D = 10,
                        max_Q=5, m=12, seasonal=True, 
                        error_action = 'warn', trace = True,
                        random_state = 20, n_fits = 50)

In [ ]:
arima_model.summary()

In [ ]:
prediction = pd.DataFrame(arima_model.predict(n_periods = 52), index=test.index)
prediction.columns = ['prediction_sales']
prediction.head()

In [ ]:
plt.figure(figsize=(8,5))
plt.plot(train, label = 'Training')
plt.plot(test, label = 'Test')
plt.plot(prediction, label = 'Prediction')
plt.legend(loc = 'best')
plt.show()

In [ ]:
test['predicted_sales'] = prediction
r2_score(test['value'], test['predicted_sales'])

### 10576

In [ ]:
oh_five_seven_six.plot(figsize = (18,6))
plt.xlabel('Time', fontsize=14);
plt.title('Median Sale Prices in 10576')

### The Augmented Dickey-Fuller Test can be used to test for stationarity. 

In [ ]:
adf_test.should_diff(oh_five_seven_six)

In [ ]:
train = oh_five_seven_six[:212]
test = oh_five_seven_six[213:265]
plt.plot(train)
plt.plot(test)

In [ ]:
ndiffs(oh_five_seven_six)

In [ ]:
arima_model = auto_arima(train, start_p = 0, d=1, start_q = 0,
                         max_p=10, max_d=10, max_q = 10, start_P = 0,
                        D = 1, start_Q=0, max_P=10, max_D = 10,
                        max_Q=5, m=12, seasonal=True, 
                        error_action = 'warn', trace = True,
                        random_state = 20, n_fits = 50)

In [ ]:
arima_model.summary()

In [ ]:
prediction = pd.DataFrame(arima_model.predict(n_periods = 52), index=test.index)
prediction.columns = ['prediction_sales']
prediction.head()

In [ ]:
plt.figure(figsize=(8,5))
plt.plot(train, label = 'Training')
plt.plot(test, label = 'Test')
plt.plot(prediction, label = 'Prediction')
plt.legend(loc = 'best')
plt.show()

In [ ]:
test['predicted_sales'] = prediction
r2_score(test['value'], test['predicted_sales'])

### 10583

In [ ]:
five_eight_three.plot(figsize = (18,6))
plt.xlabel('Time', fontsize=14);
plt.title('Median Sale Prices in 10583')

### The Augmented Dickey-Fuller Test can be used to test for stationarity. 

In [ ]:
adf_test.should_diff(five_eight_three)

In [ ]:
train = five_eight_three[:212]
test = five_eight_three[213:265]
plt.plot(train)
plt.plot(test)

In [ ]:
ndiffs(five_eight_three)

In [ ]:
arima_model = auto_arima(train, start_p = 0, d=1, start_q = 0,
                         max_p=10, max_d=10, max_q = 10, start_P = 0,
                        D = 1, start_Q=0, max_P=10, max_D = 10,
                        max_Q=5, m=12, seasonal=True, 
                        error_action = 'warn', trace = True,
                        random_state = 20, n_fits = 50)

In [ ]:
arima_model.summary()

In [ ]:
prediction = pd.DataFrame(arima_model.predict(n_periods = 52), index=test.index)
prediction.columns = ['prediction_sales']
prediction.head()

In [ ]:
plt.figure(figsize=(8,5))
plt.plot(train, label = 'Training')
plt.plot(test, label = 'Test')
plt.plot(prediction, label = 'Prediction')
plt.legend(loc = 'best')
plt.show()

In [ ]:
test['predicted_sales'] = prediction
r2_score(test['value'], test['predicted_sales'])

### 10504

In [ ]:
five_oh_four.plot(figsize = (18,6))
plt.xlabel('Time', fontsize=14);
plt.title('Median Sale Prices in 10504')

### The Augmented Dickey-Fuller Test can be used to test for stationarity. 

In [ ]:
adf_test.should_diff(five_oh_four)

In [ ]:
train = five_oh_four[:212]
test = five_oh_four[213:265]
plt.plot(train)
plt.plot(test)

In [ ]:
ndiffs(five_oh_four)

In [ ]:
arima_model = auto_arima(train, start_p = 0, d=1, start_q = 0,
                         max_p=10, max_d=10, max_q = 10, start_P = 0,
                        D = 1, start_Q=0, max_P=10, max_D = 10,
                        max_Q=5, m=12, seasonal=True, 
                        error_action = 'warn', trace = True,
                        random_state = 20, n_fits = 50)

In [ ]:
arima_model.summary()

In [ ]:
prediction = pd.DataFrame(arima_model.predict(n_periods = 52), index=test.index)
prediction.columns = ['prediction_sales']
prediction.head()

In [ ]:
plt.figure(figsize=(8,5))
plt.plot(train, label = 'Training')
plt.plot(test, label = 'Test')
plt.plot(prediction, label = 'Prediction')
plt.legend(loc = 'best')
plt.show()

In [ ]:
test['predicted_sales'] = prediction
r2_score(test['value'], test['predicted_sales'])

### 10580

In [ ]:
five_eight_oh.plot(figsize = (18,6))
plt.xlabel('Time', fontsize=14);
plt.title('Median Sale Prices in 10580')

### The Augmented Dickey-Fuller Test can be used to test for stationarity. 

In [ ]:
adf_test.should_diff(five_eight_oh)

In [ ]:
train = five_eight_oh[:212]
test = five_eight_oh[213:265]
plt.plot(train)
plt.plot(test)

In [ ]:
ndiffs(five_eight_oh)

In [ ]:
arima_model = auto_arima(train, start_p = 0, d=1, start_q = 0,
                         max_p=10, max_d=10, max_q = 10, start_P = 0,
                        D = 1, start_Q=0, max_P=10, max_D = 10,
                        max_Q=5, m=12, seasonal=True, 
                        error_action = 'warn', trace = True,
                        random_state = 20, n_fits = 50)

In [ ]:
arima_model.summary()

In [ ]:
prediction = pd.DataFrame(arima_model.predict(n_periods = 52), index=test.index)
prediction.columns = ['prediction_sales']
prediction.head()

In [ ]:
plt.figure(figsize=(8,5))
plt.plot(train, label = 'Training')
plt.plot(test, label = 'Test')
plt.plot(prediction, label = 'Prediction')
plt.legend(loc = 'best')
plt.show()

In [ ]:
test['predicted_sales'] = prediction
r2_score(test['value'], test['predicted_sales'])

### 11797

In [ ]:
seven_nine_seven.plot(figsize = (18,6))
plt.xlabel('Time', fontsize=14);
plt.title('Median Sale Prices in 11797')

### The Augmented Dickey-Fuller Test can be used to test for stationarity. 

In [ ]:
adf_test.should_diff(seven_nine_seven)

In [ ]:
train = seven_nine_seven[:212]
test = seven_nine_seven[213:265]
plt.plot(train)
plt.plot(test)

In [ ]:
ndiffs(seven_nine_seven)

In [ ]:
arima_model = auto_arima(train, start_p = 0, d=1, start_q = 0,
                         max_p=10, max_d=10, max_q = 10, start_P = 0,
                        D = 1, start_Q=0, max_P=10, max_D = 10,
                        max_Q=5, m=12, seasonal=True, 
                        error_action = 'warn', trace = True,
                        random_state = 20, n_fits = 50)

In [ ]:
arima_model.summary()

In [ ]:
prediction = pd.DataFrame(arima_model.predict(n_periods = 52), index=test.index)
prediction.columns = ['prediction_sales']
prediction.head()

In [ ]:
plt.figure(figsize=(8,5))
plt.plot(train, label = 'Training')
plt.plot(test, label = 'Test')
plt.plot(prediction, label = 'Prediction')
plt.legend(loc = 'best')
plt.show()

In [ ]:
test['predicted_sales'] = prediction
r2_score(test['value'], test['predicted_sales'])

### 11976

In [ ]:
nine_seven_six.plot(figsize = (18,6))
plt.xlabel('Time', fontsize=14);
plt.title('Median Sale Prices in 11976')

### The Augmented Dickey-Fuller Test can be used to test for stationarity. 

In [ ]:
adf_test.should_diff(nine_seven_six)

In [ ]:
train = nine_seven_six[:212]
test = nine_seven_six[213:265]
plt.plot(train)
plt.plot(test)

In [ ]:
ndiffs(nine_seven_six)

In [ ]:
arima_model = auto_arima(train, start_p = 0, d=1, start_q = 0,
                         max_p=10, max_d=10, max_q = 10, start_P = 0,
                        D = 1, start_Q=0, max_P=10, max_D = 10,
                        max_Q=5, m=12, seasonal=True, 
                        error_action = 'warn', trace = True,
                        random_state = 20, n_fits = 50)

In [ ]:
arima_model.summary()

In [ ]:
prediction = pd.DataFrame(arima_model.predict(n_periods = 52), index=test.index)
prediction.columns = ['prediction_sales']
prediction.head()

In [ ]:
plt.figure(figsize=(8,5))
plt.plot(train, label = 'Training')
plt.plot(test, label = 'Test')
plt.plot(prediction, label = 'Prediction')
plt.legend(loc = 'best')
plt.show()

In [ ]:
test['predicted_sales'] = prediction
r2_score(test['value'], test['predicted_sales'])

### 10506

In [ ]:
five_oh_six.plot(figsize = (18,6))
plt.xlabel('Time', fontsize=14);
plt.title('Median Sale Prices in 10506')

### The Augmented Dickey-Fuller Test can be used to test for stationarity. 

In [ ]:
adf_test.should_diff(five_oh_six)

In [ ]:
train = five_oh_six[:212]
test = five_oh_six[213:265]
plt.plot(train)
plt.plot(test)

In [ ]:
ndiffs(five_oh_six)

In [ ]:
arima_model = auto_arima(train, start_p = 0, d=1, start_q = 0,
                         max_p=10, max_d=10, max_q = 10, start_P = 0,
                        D = 1, start_Q=0, max_P=10, max_D = 10,
                        max_Q=5, m=12, seasonal=True, 
                        error_action = 'warn', trace = True,
                        random_state = 20, n_fits = 50)

In [ ]:
arima_model.summary()

In [ ]:
prediction = pd.DataFrame(arima_model.predict(n_periods = 52), index=test.index)
prediction.columns = ['prediction_sales']
prediction.head()

In [ ]:
plt.figure(figsize=(8,5))
plt.plot(train, label = 'Training')
plt.plot(test, label = 'Test')
plt.plot(prediction, label = 'Prediction')
plt.legend(loc = 'best')
plt.show()

In [ ]:
test['predicted_sales'] = prediction
r2_score(test['value'], test['predicted_sales'])

### 10538

In [ ]:
five_three_eight.plot(figsize = (18,6))
plt.xlabel('Time', fontsize=14);
plt.title('Median Sale Prices in 10538')

### The Augmented Dickey-Fuller Test can be used to test for stationarity. 

In [ ]:
adf_test.should_diff(five_three_eight)

In [ ]:
train = five_three_eight[:212]
test = five_three_eight[213:265]
plt.plot(train)
plt.plot(test)

In [ ]:
ndiffs(five_three_eight)

In [ ]:
arima_model = auto_arima(train, start_p = 0, d=1, start_q = 0,
                         max_p=10, max_d=10, max_q = 10, start_P = 0,
                        D = 1, start_Q=0, max_P=10, max_D = 10,
                        max_Q=5, m=12, seasonal=True, 
                        error_action = 'warn', trace = True,
                        random_state = 20, n_fits = 50)

In [ ]:
arima_model.summary()

In [ ]:
prediction = pd.DataFrame(arima_model.predict(n_periods = 52), index=test.index)
prediction.columns = ['prediction_sales']
prediction.head()

In [ ]:
plt.figure(figsize=(8,5))
plt.plot(train, label = 'Training')
plt.plot(test, label = 'Test')
plt.plot(prediction, label = 'Prediction')
plt.legend(loc = 'best')
plt.show()

In [ ]:
test['predicted_sales'] = prediction
r2_score(test['value'], test['predicted_sales'])

In [ ]:
gtrends_diff = gtrends.diff(periods=1)

gtrends_diff.plot(figsize=(18,6))
plt.xlabel('Year', fontsize=14);

In [ ]:
df.corr()

# Step 6: Interpreting Results